In [19]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_ta as ta
import math
import time

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras.layers import LSTM, GRU
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.python.keras.models import load_model
from datetime import date
from datetime import timedelta

training_start_time = time.time()

today = date.today()
data_end_date = today.strftime("%Y-%m-%d")
start_date = date.today() - timedelta(days=1095)
data_start_date = start_date.strftime("%Y-%m-%d")
df_data = yf.download('XRP-USD', start=data_start_date, end=data_end_date)

[*********************100%***********************]  1 of 1 completed


In [20]:
def move_close_to_first_column(df_data):
    df_data.drop(columns=['Adj Close'], inplace=True)
    close_price_column = df_data.pop('Close')
    df_data.insert(0, 'Close', close_price_column)
    return df_data

In [21]:
def prepare_ti_dataset(df_data):
    df_data.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)
    CustomStrategy = ta.Strategy(
        name="Technical indicators",
        description="SMA 5, 10, 20, 30, 50, WMA 10, 20, 30, RSI, EMA, MACD",
        ta=[
            {"kind": "sma", "length": 5},
            {"kind": "sma", "length": 10},
            {"kind": "sma", "length": 20},
            {"kind": "sma", "length": 30},
            {"kind": "sma", "length": 50},
            {"kind": "wma", "length": 10},
            {"kind": "wma", "length": 20},
            {"kind": "wma", "length": 30},
            {"kind": "rsi"},
            {"kind": "ema"},
            {"kind": "macd", "fast": 8, "slow": 21},
        ]
    )

    df_data.ta.strategy(CustomStrategy)
    df_data = df_data.dropna()
    return df_data

In [22]:
def prepare_combined_dataset(df_data):
    df_data.drop(columns=['Open', 'Volume'], inplace=True)
    CustomStrategy = ta.Strategy(
        name="Test",
        description="SMA 50,200, BBANDS, RSI, MACD and Volume SMA 20",
        ta=[
            {"kind": "sma", "length": 5},
            {"kind": "wma", "length": 10},
            {"kind": "ema"},
        ]
    )

    df_data.ta.strategy(CustomStrategy)
    df_data = df_data.dropna()
    return df_data

In [23]:
df_data = move_close_to_first_column(df_data)
# df_data = prepare_ti_dataset(df_data)
# df_data = prepare_combined_dataset(df_data)

In [24]:
prediction_days = int(len(df_data)*0.2)

df_train = df_data[:len(df_data) - prediction_days]
df_test = df_data[len(df_data)-prediction_days:]

actual_test_df = df_test

columns_count = len(df_data.columns)
df_train = df_train.values.reshape(-1, columns_count)
df_test = df_test.values.reshape(-1, columns_count)

# Normalize data
scaler_train = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler_train.fit_transform(df_train)

scaler_test = MinMaxScaler(feature_range=(0, 1))
scaled_test = scaler_test.fit_transform(df_test)

result_scaler = MinMaxScaler(feature_range=(0, 1))
result_scaler.fit_transform(df_test[:, 0:1])

array([[7.91307255e-04],
       [0.00000000e+00],
       [2.28791011e-03],
       [1.06459288e-02],
       [1.46360565e-02],
       [6.27379669e-02],
       [6.49187847e-02],
       [8.71824801e-02],
       [1.07414361e-01],
       [8.52451223e-02],
       [8.88028772e-02],
       [9.11851812e-02],
       [7.38498599e-02],
       [1.03934298e-01],
       [1.44987882e-01],
       [1.24395002e-01],
       [1.19009046e-01],
       [1.69333747e-01],
       [1.33156055e-01],
       [1.54116252e-01],
       [1.38185142e-01],
       [1.43827194e-01],
       [1.34052328e-01],
       [1.45869454e-01],
       [1.05626069e-01],
       [1.30389608e-01],
       [1.46138123e-01],
       [1.37437935e-01],
       [1.42597196e-01],
       [1.40708129e-01],
       [1.14748122e-01],
       [1.01917184e-01],
       [1.25173673e-01],
       [1.12890583e-01],
       [8.01761269e-02],
       [8.05518883e-02],
       [8.34127299e-02],
       [6.50971885e-02],
       [5.75262859e-02],
       [8.00523326e-02],


In [25]:
# Dataset Generator for LSTM
def dataset_generator(dataset, look_back=5):
    dataX, dataY = [], []
    
    for i in range(len(dataset) - look_back):
        window_size_x = dataset[i:(i + look_back), 1:]
        dataX.append(window_size_x)
        dataY.append(dataset[i + look_back, 0]) # this is the label or actual y-value
    return np.array(dataX), np.array(dataY)

In [26]:
look_back_period = 15
trainX, trainY = dataset_generator(scaled_train, look_back_period)
testX, testY = dataset_generator(scaled_test, look_back_period)

In [27]:
def get_lstm_model(timesteps_count, features_count):
    regressor = Sequential()
    regressor.add(LSTM(units = 128, activation = 'relu',return_sequences=True, input_shape = (timesteps_count, features_count)))
    regressor.add(Dropout(0.2))

    # Adding a second LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 64, input_shape = (timesteps_count, features_count)))
    regressor.add(Dropout(0.2))

    # Adding the output layer
    regressor.add(Dense(units = 1))
    regressor.summary()
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return regressor

In [28]:
timesteps_count = trainX.shape[1]
features_count = trainX.shape[2]
regressor = get_lstm_model(timesteps_count, features_count)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 15, 128)           68096     
_________________________________________________________________
dropout_2 (Dropout)          (None, 15, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 117,569
Trainable params: 117,569
Non-trainable params: 0
_________________________________________________________________


In [ ]:
checkpoint_path = 'lstm-checkpoint.hdf5'
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')


earlystopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
callbacks = [checkpoint, earlystopping]
history = regressor.fit(trainX, trainY, batch_size = 32, epochs = 600, verbose=1, shuffle=False, validation_data=(testX, testY), callbacks=callbacks)

Epoch 1/600
27/27 [==============================] - 5s 47ms/step - loss: 0.0086 - val_loss: 0.0409

Epoch 00001: val_loss improved from inf to 0.04092, saving model to lstm-checkpoint.hdf5
Epoch 2/600
27/27 [==============================] - 1s 34ms/step - loss: 0.0202 - val_loss: 0.0283

Epoch 00002: val_loss improved from 0.04092 to 0.02832, saving model to lstm-checkpoint.hdf5
Epoch 3/600
27/27 [==============================] - 1s 32ms/step - loss: 0.0179 - val_loss: 0.0108

Epoch 00003: val_loss improved from 0.02832 to 0.01082, saving model to lstm-checkpoint.hdf5
Epoch 4/600
27/27 [==============================] - 1s 31ms/step - loss: 0.0083 - val_loss: 0.0194

Epoch 00004: val_loss did not improve from 0.01082
Epoch 5/600
27/27 [==============================] - 1s 32ms/step - loss: 0.0132 - val_loss: 0.0100

Epoch 00005: val_loss improved from 0.01082 to 0.00997, saving model to lstm-checkpoint.hdf5
Epoch 6/600
27/27 [==============================] - 1s 30ms/step - loss: 0.

In [ ]:
training_time = time.time() - training_start_time
predicting_start_time = time.time()

In [ ]:
# Plot line graph to show Loss Numbers relative to the epoch
plt.figure(figsize=(16,7))
plt.plot(history.history['loss'], label='Loss function value for train data')

plt.plot(history.history['val_loss'], label='Loss function value for test data')
plt.legend()
plt.show()

In [ ]:
model_from_saved_checkpoint = load_model(checkpoint_path)

# Prediction for the existing data
predicted_price = model_from_saved_checkpoint.predict(testX)

predicted_price = result_scaler.inverse_transform(predicted_price.reshape(-1, 1))
test_actual = result_scaler.inverse_transform(testY.reshape(-1, 1))

df_data_len = len(df_data)
date_range = df_data[df_data_len-len(predicted_price):].index

In [ ]:
# Predicting next day price
x_input = scaled_test[-look_back_period:, 1:]
x_input = x_input.reshape(1, look_back_period, features_count)

prediction_for_next_day = model_from_saved_checkpoint.predict(x_input)
next_day_price = result_scaler.inverse_transform(prediction_for_next_day.reshape(-1, 1))
next_day_price = next_day_price.reshape(-1)[0]

date_range = actual_test_df.index.tolist()
date_range.append(date.today())

predicted_prices = predicted_price.reshape(-1).tolist()
predicted_prices_with_next_day = predicted_prices
predicted_prices_with_next_day.append(next_day_price)

predicted_prices_for_plot = np.empty_like(date_range)
predicted_prices_for_plot[:] = np.nan
predicted_prices_for_plot[look_back_period:] = predicted_prices_with_next_day

actual_prices = actual_test_df['Close'].values.tolist()
actual_prices.append(np.nan)

plt.figure(figsize=(16,7))
plt.grid(True)
plt.xlabel('Date')
plt.ylabel('Closing price')

plt.plot(date_range, predicted_prices_for_plot, 'r', marker='.', label='Predicted price')

plt.plot(date_range, actual_prices, marker='.', label='Actual price')

plt.legend()
plt.show()

In [ ]:
predicting_time = time.time() - predicting_start_time

# report performance
model_predictions = predicted_price
testing_data = test_actual
mape = np.mean(np.abs(np.array(model_predictions) - np.array(testing_data))/np.abs(testing_data))
mape_percentage = mape * 100
model_accuracy = 100 - mape_percentage
rmse = math.sqrt(mean_squared_error(test_actual, predicted_price))

print('MAPE: %.2f %%' % mape_percentage)
print('Model accuracy: %.2f %%' % model_accuracy)
print('RMSE: %.3f' % rmse)
print('Training time: %.2f in seconds' % training_time)
print('Predicting time: %.2f in seconds' % predicting_time)

In [ ]:
from tqdm.notebook import tqdm

results = []
print(' Computing LSTM feature importance...')

# COMPUTE BASELINE (NO SHUFFLE)
oof_preds = model_from_saved_checkpoint.predict(testX, verbose=0).squeeze() 
baseline_mae = np.mean(np.abs( oof_preds-testY ))
results.append({'feature':'BASELINE','mae':baseline_mae})
COLS = list(df_data.loc[:, df_data.columns != 'Close'].columns)

for k in tqdm(range(len(COLS))):           
    # SHUFFLE FEATURE K
    save_col = testX[:,:,k].copy()
    np.random.shuffle(testX[:,:,k])

    # COMPUTE OOF MAE WITH FEATURE K SHUFFLED
    oof_preds = model_from_saved_checkpoint.predict(testX, verbose=0).squeeze() 
    mae = np.mean(np.abs( oof_preds-testY ))
    results.append({'feature':COLS[k],'mae':mae})
    testX[:,:,k] = save_col
    

In [ ]:
# DISPLAY LSTM FEATURE IMPORTANCE
df = pd.DataFrame(results)
df = df.sort_values('mae')
plt.figure(figsize=(10,5))
plt.barh(np.arange(len(COLS)+1),df.mae)
plt.yticks(np.arange(len(COLS)+1),df.feature.values)
plt.title('Feature importance',size=16)
plt.ylim((-1,len(COLS)+1))
plt.plot([baseline_mae,baseline_mae],[-1,len(COLS)+1], '--', color='orange',
         label=f'Baseline OOF\nMAE={baseline_mae:.3f}')
plt.ylabel('Feature',size=14)
plt.legend()
plt.show()